In [6]:
import numpy as np
from scipy.stats import binom

t = np.array([7, 7, 1, 3, 2, 1, 5, 1, 4, 0, 2, 3, 3, 1, 2, 2, 3,
              1, 4, 0, 8, 3, 5, 3, 0, 0, 7, 1, 1, 3, 1, 3, 2, 4, 1, 6, 2, 2, 4, 1, 3, 1, 1, 2, 7, 3, 3, 2, 2, 2])

In [10]:
# E-шаг
def e_step(data, M, mu1_step, mu2_step, pi1_step):
    pi2_step = 1 - pi1_step
    r_i1 = binom.pmf(data, M, mu1_step) * pi1_step / (binom.pmf(data, M, mu1_step) * pi1_step 
                                                    + binom.pmf(data, M, mu2_step) * pi2_step)
    r_i2 = binom.pmf(data, M, mu2_step) * pi2_step / (binom.pmf(data, M, mu1_step) * pi1_step 
                                                    + binom.pmf(data, M, mu2_step) * pi2_step)
    return r_i1, r_i2

In [11]:
# M-шаг
def m_step(data, M, r1, r2):
    pi1_new = sum(r1) / len(data)
    pi2_new = 1 - pi1_new
    mu1_new = sum(data * r1) / (M * sum(r1))
    mu2_new = sum(data * r2) / (M * sum(r2))

    return pi1_new, pi2_new, mu1_new, mu2_new

In [34]:
# Задаем случайным образом параметры и останавливаем процесс, когда оценки параметров перестают изменяться значимо.
def em_coins(data, M = 10, mu1 = None, mu2 = None, pi1 = None, eps = .0005):
    mu1 = mu1 or np.random.random()
    mu2 = mu2 or np.random.random()
    pi1 = pi1 or np.random.random()
    pi2 = 1 - pi1
    k = 0
    print("Начальные параметры: pi1 = %0.3f,  pi2 = %0.3f,  mu1 = %0.3f,  mu2 = %0.3f\n" % (pi1, pi2, mu1, mu2))
    print("№ Шага\t  pi1     pi2     mu1     mu2")
    while True:
        r1, r2 = e_step(data, M, mu1, mu2, pi1)
        pi1_new, pi2_new, mu1_new, mu2_new = m_step(data, M, r1, r2)
        print("#%d:\t  %0.3f   %0.3f   %0.3f   %0.3f" % (k, pi1_new, pi2_new, mu1_new, mu2_new))
        k = k + 1
        if np.sqrt((mu1 - mu1_new)**2 + (mu2 - mu2_new)**2 + 2 * (pi1 - pi1_new)**2) < eps:
            break
            
        mu1, mu2, pi1 = mu1_new, mu2_new, pi1_new
    print("Конец алгоритма после %d шага" % k)

In [35]:
em_coins(t)

Начальные параметры: pi1 = 0.462,  pi2 = 0.538,  mu1 = 0.877,  mu2 = 0.063

№ Шага	  pi1     pi2     mu1     mu2
#0:	  0.169   0.831   0.635   0.196
#1:	  0.161   0.839   0.629   0.201
#2:	  0.158   0.842   0.632   0.202
#3:	  0.156   0.844   0.634   0.203
#4:	  0.155   0.845   0.636   0.203
#5:	  0.154   0.846   0.637   0.203
#6:	  0.154   0.846   0.637   0.203
#7:	  0.153   0.847   0.638   0.203
#8:	  0.153   0.847   0.638   0.204
Конец алгоритма после 9 шага
